In [1]:
using Random
using Distributions
using LinearAlgebra
include("../Inversion/Plot.jl")
include("../Inversion/KalmanInversion.jl")
include("../Inversion/GMNVI.jl")

Gaussian_mixture_NGFVI (generic function with 1 method)

In [2]:
mutable struct Setup_Param{IT<:Int}
    θ_names::Array{String,1}
    N_θ::IT
    N_y::IT
end

function Setup_Param(N_θ::IT, N_y::IT) where {IT<:Int}
    return Setup_Param(["θ"], N_θ, N_y)
end

Setup_Param

## Gaussian Mixture Kalman Inversion

## Circle shape likelihood function

We will explore the question with the following inverse problem 

$$
\begin{align*}
y = 
\Bigl(
\theta_{(1)}^2 +  \theta_{(2)}^2
\Bigr)
+\eta
\qquad 
y = 1
\qquad
\Sigma_{\eta} = 
\begin{bmatrix}
0.3^2 
\end{bmatrix} 
\end{align*}
$$


We focus on the noninformative uniform prior, and hence we are sampling $e^{-\frac{1}{2}(y-\theta_{(1)}^2 -  \theta_{(2)}^2)^T\Sigma_{\eta}^{-1}(y-\theta_{(1)}^2 -  \theta_{(2)}^2)}$.
     

In [3]:
function func_G(θ)
    θ₁, θ₂ = θ
    return [θ₁^2 + θ₂^2;]
end


function log_circle_shape(θ, y, ση)
    θ₁, θ₂ = θ
    return -(y[1] - func_G(θ)[1])^2/(2ση^2) 
end

function V_circle_shape(θ, y, ση)
    θ₁, θ₂ = θ    
    V   = (y[1] - θ₁^2 - θ₂^2)^2/(2ση^2)
    ∇V  = [ 2(y[1] - θ₁^2 - θ₂^2)*(-2θ₁)/(2ση^2) ;   2(y[1] - θ₁^2 - θ₂^2)*(-2θ₂)/(2ση^2)]
    ∇²V = [ (-4*(y[1] - θ₁^2 - θ₂^2) + 8θ₁^2)/(2ση^2)   8*θ₁*θ₂/(2ση^2);   8*θ₁*θ₂/(2ση^2)   (-4*(y[1] - θ₁^2 - θ₂^2) + 8θ₂^2)/(2ση^2)] 
    return V, ∇V, ∇²V
    
end

function forward(s_param::Setup_Param, θ)
    return func_G(θ)
end




function forward(s_param::Setup_Param, θ)
    return func_G(θ)
end



forward (generic function with 1 method)

In [4]:
using ForwardDiff
using KernelDensity
function circle_shape(μ0 = [0.0;0.0];Run_GMKI = false, Run_GMNVI = false)

    σ0 = 1.0
    Σ0 = [σ0^2 0.0;0.0  σ0^2]

    ση = 0.3
    Σ_η = [ση^2;;]
    y = [1.0;]

    

    N_y, N_θ = 1, 2
    # compute posterior distribution by GMKI
    update_freq = 1
    N_modes = 10
    θ0_w  = ones(N_modes)/N_modes
    θ0_mean, θθ0_cov  = zeros(N_modes, N_θ), zeros(N_modes, N_θ, N_θ)


    Random.seed!(111);
    for m = 1:N_modes
        θ0_mean[m, :]    .= rand(MvNormal(zeros(N_θ), Σ0)) + μ0
        θθ0_cov[m, :, :] .= Σ0
    end

    s_param = Setup_Param(N_θ, N_y)

    
    if Run_GMKI
        N_iter = 30
        adapt_α = false
        trunc_α = 1.0e-3
        Δt = 5e-1
        mixture_power_sampling_method = "random-sampling" # "random-sampling-derivatives"
        gmkiobj = GMKI_Run(s_param, forward, θ0_w, θ0_mean, θθ0_cov, y, Σ_η, Δt, update_freq, N_iter; unscented_transform = "modified-2n+1", adapt_α = adapt_α, trunc_α = trunc_α, mixture_power_sampling_method = "random-sampling")
    end
    
    if Run_GMNVI
        N_iter = 1000
        Δt = 1.0e-2
        gaussain_mixture_V = θ -> V_circle_shape(θ, y, ση)
        T = N_iter * Δt
   
        gmnviobj = GMNVI_Run(
            gaussain_mixture_V, 
            T,
            N_iter,
            θ0_w, θ0_mean, θθ0_cov;
            sqrt_matrix_type = "Cholesky",
            quadrature_type_GM = "mean_point",
            quadrature_type = "mean_point",
            diagonal_covariance = false,
            Hessian_correct_GM = false)
        
    end

    

    # visualization 
    x_min, x_max = -4.0, 4.0
    y_min, y_max = -4.0, 4.0
    Nx, Ny = 2000, 2000
    xx = LinRange(x_min, x_max, Nx)
    yy = LinRange(y_min, y_max, Ny)
    X,Y = repeat(xx, 1, Ny), repeat(yy, 1, Nx)' 
    
    Z_ref = copy(X)
    for i = 1:Nx
        for j = 1:Ny
            temp = [X[i,j] ; Y[i,j]]
            Z_ref[i,j] = exp(log_circle_shape(temp, y, ση))
        end
    end
    Z_ref /= ( sum(Z_ref)*(xx[2]-xx[1])*(yy[2]-yy[1]) )
    
    
    
    
    
    # Visualize different iterations
    for iter_percentage  in [0, 0.25, 0.5, 0.75, 1]
        nrows, ncols = 1, 3
        fig, ax = PyPlot.subplots(nrows=nrows, ncols=ncols, sharex=false, sharey=false, figsize=(15,5))
        
        color_lim = (minimum(Z_ref), maximum(Z_ref))
        ax[1].pcolormesh(X, Y, Z_ref, cmap="viridis", clim=color_lim)
        ax[1].set_xlabel(L"\theta_{(1)}")
        ax[1].set_ylabel(L"\theta_{(2)}")
        ax[1].set_title("Reference")
        
        
        
        
    
        if Run_GMKI
            iter = floor(Int64, iter_percentage*(length(gmkiobj.θ_mean)-1)) + 1
            Z = zeros(N_modes, Nx, Ny)
            # plot GMKI results 
            for i =1:N_modes
                gmki_θ_mean = gmkiobj.θ_mean[iter][i, :]
                gmki_θθ_cov = gmkiobj.θθ_cov[iter][i, :, :]
                _,_, Z[i, :, :] = Gaussian_2d(gmki_θ_mean, gmki_θθ_cov, Nx, Ny, x_min, x_max, y_min, y_max) 
                Z[i,:,:] *= exp(gmkiobj.logθ_w[iter][i])

            end
            Z_GMKI = sum(Z, dims=1)[1,:,:]
            Z_GMKI /= (sum(Z_GMKI)*(xx[2]-xx[1])*(yy[2]-yy[1]) )
            ax[2].pcolormesh(X, Y, Z_GMKI, cmap="viridis", clim=color_lim)
            @info "GMKI L₁ error = ", sum(abs.(Z_GMKI - Z_ref))/sum(abs.(Z_ref))


            # linestyles = ["o"; "x"; "s"]
            for i =1:N_modes
                gmki_θ_mean = gmkiobj.θ_mean[iter][i, :]
                gmki_θθ_cov = gmkiobj.θθ_cov[iter][i, :, :]
                linestyle = "o" 
                ax[2].scatter([gmki_θ_mean[1];], [gmki_θ_mean[2];], marker=linestyle, color="C"*string(i), facecolors=(linestyle=="x" ? nothing : "none"))
            end
        
            ax[2].set_xlabel(L"\theta_{(1)}")
            ax[2].set_ylabel(L"\theta_{(2)}")
            ax[2].set_title("GMKI")
        end
        
        if Run_GMNVI
            iter = floor(Int64, iter_percentage*(length(gmnviobj.x_mean)-1)) + 1
            
            Z = zeros(N_modes, Nx, Ny)
            # plot GMNVI results 
            for i =1:N_modes
                gmnvi_θ_mean = gmnviobj.x_mean[iter][i, :]
                gmnvi_θθ_cov = gmnviobj.xx_cov[iter][i, :, :]
                _,_, Z[i, :, :] = Gaussian_2d(gmnvi_θ_mean, gmnvi_θθ_cov, Nx, Ny, x_min, x_max, y_min, y_max) 
                Z[i,:,:] *= exp(gmnviobj.logx_w[iter][i])

            end
            Z_GMNVI = sum(Z, dims=1)[1,:,:]
            Z_GMNVI /= (sum(Z_GMNVI)*(xx[2]-xx[1])*(yy[2]-yy[1]) )
            
            ax[3].pcolormesh(X, Y, Z_GMNVI, cmap="viridis", clim=color_lim)
            @info "GMNVI L₁ error = ", sum(abs.(Z_GMNVI - Z_ref))/sum(abs.(Z_ref))


            # linestyles = ["o"; "x"; "s"]
            for i =1:N_modes
                gmnvi_θ_mean = gmnviobj.x_mean[iter][i, :]
                gmnvi_θθ_cov = gmnviobj.xx_cov[iter][i, :, :]
                linestyle = "o" 
                ax[3].scatter([gmnvi_θ_mean[1];], [gmnvi_θ_mean[2];], marker=linestyle, color="C"*string(i), facecolors=(linestyle=="x" ? nothing : "none"))
            end

            ax[3].set_xlabel(L"\theta_{(1)}")
            ax[3].set_ylabel(L"\theta_{(2)}")
            ax[3].set_title("GMNVI")
        end
        
        
        fig.tight_layout()
        fig.savefig("2D-circle-shape-density-"*string(iter_percentage)*".png")


    end
    
end





circle_shape (generic function with 2 methods)

In [ ]:
circle_shape([0.0;0.0]; Run_GMKI=true, Run_GMNVI=true);

[ Info: Start GMKI on the mean-field stochastic dynamical system for Bayesian inference 
[ Info: ("iter = ", 3, " / ", 30)
[ Info: ("iter = ", 6, " / ", 30)
[ Info: ("iter = ", 9, " / ", 30)
[ Info: ("iter = ", 12, " / ", 30)
[ Info: ("iter = ", 15, " / ", 30)
[ Info: ("iter = ", 18, " / ", 30)
[ Info: ("iter = ", 21, " / ", 30)
[ Info: ("iter = ", 24, " / ", 30)
[ Info: ("iter = ", 27, " / ", 30)
[ Info: ("iter = ", 30, " / ", 30)
[ Info: ("iter = ", 100, " / ", 1000)
[ Info: ("iter = ", 200, " / ", 1000)
[ Info: ("iter = ", 300, " / ", 1000)
[ Info: ("iter = ", 400, " / ", 1000)
[ Info: ("iter = ", 500, " / ", 1000)
[ Info: ("iter = ", 600, " / ", 1000)
[ Info: ("iter = ", 700, " / ", 1000)
[ Info: ("iter = ", 800, " / ", 1000)
[ Info: ("iter = ", 900, " / ", 1000)
[ Info: ("iter = ", 1000, " / ", 1000)
[ Info: ("GMKI L₁ error = ", 1.365425732834291)
[ Info: ("GMNVI L₁ error = ", 1.365425732834291)
